In [3]:
import json

import numpy as np
import pandas as pd

In [4]:
sim = "m12b"
sim_dir = "/Users/z5114326/Documents/simulations/"

public_snapshot_file = sim_dir + "snapshot_times_public.txt"
pub_data = pd.read_table(public_snapshot_file, comment="#", header=None, sep=r"\s+")
pub_data.columns = [
    "index",
    "scale_factor",
    "redshift",
    "time_Gyr",
    "lookback_time_Gyr",
    "time_width_Myr",
]
pub_snaps = np.array(pub_data["index"], dtype=int)

all_data_file = sim_dir + "m12i" + "/" + "m12i" + "_res7100/snapshot_times.txt"
all_data = pd.read_table(all_data_file, comment="#", header=None, sep=r"\s+")
all_data.columns = [
    "index",
    "scale_factor",
    "redshift",
    "time_Gyr",
    "lookback_time_Gyr",
    "time_width_Myr",
]

In [5]:
# fix missing snaps for P
def fix_P(P, tag, fix_first=False):
    for i in range(len(P)):
        if tag[i][-1] == 0:
            if fix_first:
                if tag[i][-2] != 0:
                    P[i][-1] = P[i][-2]
                    tag[i][-1] = tag[i][-2]
                else:
                    continue
            else:
                continue
        for j in range(len(P[0]) - 1, -1, -1):
            if tag[i][j] == 0:
                P[i][j] = P[i][j + 1]
                tag[i][j] = tag[i][j + 1]
    return P, tag

In [49]:
def get_it_done(it, at_snap=600, gcid_hld=37131255):
    resultpath = sim_dir + sim + "/gc_results/raw/it_%d/" % it
    sim_codes = sim_dir + "simulation_codes.json"
    with open(sim_codes) as sim_json:
        sim_data = json.load(sim_json)
    subs = [sim_data[sim]["halo"]]
    snap_offset = sim_data[sim]["offset"]

    at_snap = at_snap - snap_offset

    full_snap = pub_snaps - snap_offset
    snap_range = None
    fix_first = False

    p2 = 7.0
    p3 = 1.0
    kappa = 1.5

    disrupt_mode = "tidal"
    mu = 0.55
    disrupt_x = 0.67
    disrupt_y = 1.33

    results_path = sim_dir + sim + "/gc_results/raw/it_%d/" % it
    allcat_base = "allcat"
    allcat_name = allcat_base + "_s-%d_p2-%g_p3-%g.txt" % (it, p2, p3)
    fin_name = results_path + allcat_name
    gcid_name = fin_name[:-4] + "_gcid.txt"
    root_name = fin_name[:-4] + "_offset_root.txt"
    tid1_name = fin_name[:-4] + "_tideig1.txt"
    tid2_name = fin_name[:-4] + "_tideig2.txt"
    tid3_name = fin_name[:-4] + "_tideig3.txt"
    tag_name = fin_name[:-4] + "_tidtag.txt"

    z_list = all_data["redshift"][snap_offset:].values
    t_list = all_data["time_Gyr"][snap_offset:].values

    if snap_range is None:
        snap_range = len(full_snap)

    if not at_snap is None:
        assert at_snap <= full_snap[snap_range - 1]

    # load GC catalog
    hid, _, _, _, _, logm_form, z_form, _, ismpb, hid_form, snapnum_form = np.loadtxt(fin_name, unpack=True)

    # setting indices to int type
    hid = hid.astype(int)
    ismpb = ismpb.astype(int)
    hid_form = hid_form.astype(int)
    snapnum_form = snapnum_form.astype(int)

    # load GC id
    gcid, _ = np.loadtxt(gcid_name, ndmin=2, unpack=True, dtype="int64")

    # load root offset
    hid_root, idx_beg, idx_end, _, _ = np.loadtxt(root_name, ndmin=2, unpack=True, dtype="int64")

    logm_form += np.log10(mu)

    if disrupt_mode == "tidal":
        tideig1 = np.loadtxt(tid1_name)
        tideig3 = np.loadtxt(tid3_name)
        tideige = tideig1 - tideig3
        P_inverse = kappa * np.sqrt(tideige) / 150
        tag_name = fin_name[:-4] + "_tidtag.txt"
        tag = np.loadtxt(tag_name, dtype="int64")
        P_inverse, tag = fix_P(P_inverse, tag, fix_first)

    m_now = 10**logm_form
    snap_now = np.copy(snapnum_form)
    t_disrupt = -1 * np.ones(len(m_now))

    for i in range(len(hid_root)):
        for j in range(snap_range):
            # t1 = time.time()

            if not at_snap is None:
                if at_snap <= full_snap[j]:
                    snap = at_snap

            snap = full_snap[j]

            # existing and survived GCs at this snapshot
            idx_exist_gc = np.where(
                (snapnum_form[idx_beg[i] : idx_end[i]] < snap)
                & (tag[idx_beg[i] : idx_end[i], snap_range - 1] > 0)
                & (m_now[idx_beg[i] : idx_end[i]] > 0)
            )[0]

            # jump to the next if no existing GC
            if len(idx_exist_gc) == 0:
                continue
            idx_exist_gc += idx_beg[i]  # idx in the whole catalog

            # z_now = z_list[snap_now[idx_exist_gc]]
            t_now = t_list[snap_now[idx_exist_gc]]
            t_snap = t_list[snap]

            # TESTING ############################################################################################

            gc_mask = gcid == gcid_hld
            if np.any(gc_mask):
                gidx = np.where(gc_mask)[0][0]  # global index

                # check if included in idx_exist_gc
                included = gidx in idx_exist_gc
                # print(f"it={it}, i={i}, j={j}, snap={snap}, gidx={gidx}, included_in_idx_exist_gc={included}")
                # print(t_now[np.where(idx_exist_gc == gidx)[0][0]])

            ######################################################################################################

            mi = 10 ** logm_form[idx_exist_gc]
            t_tid = (
                10
                * (mi / 2e5) ** disrupt_x
                * (m_now[idx_exist_gc] / mi) ** disrupt_y
                / np.clip(P_inverse[idx_exist_gc, j], 1e-10, None)
            )

            t_iso = 1e10  # i.e., no iso disruption. 17 * (m_now[idx_exist_gc]/2e5) # in Gyr

            idx_tid = np.where(np.greater(t_iso, t_tid))[0]
            idx_iso = np.where(np.less_equal(t_iso, t_tid))[0]

            if len(idx_tid):
                # tidal dominated
                gamma = disrupt_y
                dt = t_tid[idx_tid] / gamma
                k1 = 1 - (t_snap - t_now[idx_tid]) / dt
                k1 = np.clip(k1, 0, 1)
                m_now[idx_exist_gc[idx_tid]] = m_now[idx_exist_gc[idx_tid]] * k1 ** (1 / gamma)

                idx_d = np.where((dt - (t_snap - t_now[idx_tid])) <= 0)[0]  # disrupted
                t_disrupt[idx_exist_gc[idx_tid[idx_d]]] = t_now[idx_tid[idx_d]] + dt[idx_d]

            # TESTING ############################################################################################

            # if included:
            #     print("snap:", snap, f"{np.log10(m_now[gidx])}")

            ######################################################################################################

            if len(idx_iso) > 0:
                # iso dominated
                m_now[idx_exist_gc[idx_iso]] = m_now[idx_exist_gc[idx_iso]] - (t_snap - t_now[idx_iso]) * (
                    2e5 / 17
                ) * np.ones(len(idx_iso))

            # TESTING ############################################################################################

            if included:
                gidxx = np.where(idx_exist_gc == gidx)[0][0]
                if gidxx in idx_iso:
                    tid_type = "iso"
                if gidxx in idx_tid:
                    tid_type = "tid"
                # print()
                print("snap:", snap, f"{np.log10(m_now[gidx])}", tid_type, t_tid[gidxx] / t_iso)

            ######################################################################################################

            # update snap_now
            snap_now[idx_exist_gc] = snap * np.ones(len(idx_exist_gc), dtype=int)

            if not at_snap is None:
                if at_snap <= full_snap[j]:
                    break

    idx_valid = np.where(
        (snapnum_form < full_snap[snap_range - 1]) & (tag[:, snap_range - 1] > 0) & (m_now > 0)
    )[0]

    m_out = -1 * np.ones(len(m_now))
    m_out[idx_valid] = np.log10(m_now[idx_valid])

In [55]:
get_it_done(it=27)

snap: 171 4.816603452932544 tid 1.2425875822355471e-09
snap: 213 4.777509643095554 tid 1.193338674545901e-09
snap: 276 4.7210356362996215 tid 1.3427950236451053e-09
snap: 293 4.703509944529763 tid 1.1295339634511052e-09
snap: 311 4.672687249422057 tid 6.966727998365752e-10
snap: 331 4.633308183691408 tid 6.146163926991719e-10
snap: 355 4.567389087573301 tid 4.5752803142205394e-10
snap: 381 4.53540477661283 tid 9.652209343829566e-10
snap: 411 4.493576138376252 tid 8.547424359898352e-10
snap: 445 4.322897637437996 iso 1.7065019617925548
snap: 485 4.253522768071408 tid 6.862851204794117e-10
snap: 533 4.0631150202398105 tid 3.44603351115986e-10
snap: 589 3.897119704325504 tid 4.2583816779634035e-10
snap: 590 3.8967666612955436 tid 2.7338124941302796e-10
snap: 591 3.89641907055618 tid 2.7743328094779195e-10
snap: 592 3.895899489034399 tid 1.8569880523299666e-10
snap: 593 3.8954803347589686 tid 2.300480881392828e-10
snap: 594 3.895229498828438 tid 3.842819276536732e-10
snap: 595 3.8949611543

In [53]:
get_it_done(it=30)

snap: 171 4.681808486152878 tid 1.0167800330222578e-09
snap: 213 4.632585319519379 tid 9.622466547647177e-10
snap: 276 4.558425782383328 tid 1.0496883392282824e-09
snap: 293 4.534529066385584 tid 8.364272790342825e-10
snap: 311 4.4912857703834215 tid 5.059228407244386e-10
snap: 331 4.4333767864282 tid 4.296750856469783e-10
snap: 355 4.327578421666444 tid 3.022100820891098e-10
snap: 381 4.27079763250175 tid 5.642590677021502e-10
snap: 411 4.188871067074052 tid 4.6313517259258976e-10
snap: 445 4.188871067028346 tid 0.8178036546943468
snap: 485 4.100623320439122 tid 5.546886050842369e-10
snap: 533 3.8179796786675495 tid 2.628834411814212e-10
snap: 589 3.4324724569831497 tid 2.449145464053658e-10
snap: 590 3.4312686023580383 tid 8.027639939833511e-11
snap: 591 3.4300802657053624 tid 8.12542600138915e-11
snap: 592 3.428298201422462 tid 5.424734643679161e-11
snap: 593 3.42685554165435 tid 6.694357722764907e-11
snap: 594 3.4259900370963443 tid 1.1147538623151234e-10
snap: 595 3.42506231153319

- **Tidal regime (`tid`)**: cluster affected by galaxy tides; higher mass → more mass stripped; lower mass → tidal effect small.  

- **Isolated regime (`iso`)**: tidal forces negligible; mass loss very slow, dominated by internal processes.  
- **it027 vs it030**:  
  - it027 heavier at snap 411 → classified `iso` by timescale, loses noticeable mass.  
  - it030 lighter → still `tid` but absolute mass loss tiny, effectively no change.  
- **Key point**: `tid` label doesn’t guarantee measurable mass loss; actual mass loss depends on cluster mass and timestep.
